# Combining LCEL Cahins

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [1]:
!pip install -U langchain-community langchain langchain-huggingface huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
HUGGINGFACEHUB_API_TOKEN= userdata.get('HUGGING_FACE_API_KEY')

from huggingface_hub import login
login(token = HUGGINGFACEHUB_API_TOKEN)

from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
import transformers, torch

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03
)
pipe.tokenizer.pad_token = pipe.tokenizer.eos_token  # Set pad_token to eos_token

model_kwargs = {'temperature':0.3}
model = HuggingFacePipeline(pipeline = pipe, model_kwargs = model_kwargs)

chatModel= ChatHuggingFace(llm=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [5]:
prompt = ChatPromptTemplate.from_template("tell me a achivemet and award  fact about {soccer_player}")

output_parser = StrOutputParser()

In [6]:
chain = prompt | chatModel | output_parser

In [7]:
chain.invoke({"soccer_player": "Ronaldo"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me a achivemet and award  fact about Ronaldo<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere's an achievement and award fact about Cristiano Ronaldo:\n\n**Achievement:** \n- **First player to win the Ballon d'Or in three consecutive years (2008, 2009, and 2013)**\n\n**Award:** \n- **Most successful player in UEFA Champions League history with 14 titles**\n\nRonaldo is widely regarded as one of the greatest soccer players of all time."

## Use of .bind() to add arguments to a Runnable in a LCEL Chain
* For example, we can add an argument to stop the model response when it reaches the word "Ronaldo":

In [8]:
chain = prompt | chatModel.bind(stop=["Ronaldo"]) | output_parser

In [9]:
chain.invoke({"soccer_player": "Ronaldo"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me a achivemet and award  fact about Ronaldo<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHere's an achievement and award fact about Cristiano Ronaldo:\n\n**Achievement:** \n- **First player to win the Ballon d'Or in three consecutive years (2008, 2009, and 2013)**\n\n**Award:** \n- **Most successful player in UEFA Champions League history with 14 titles**\n\nRonaldo is widely regarded as one of the greatest soccer players of all time."

In [10]:
prompt = ChatPromptTemplate.from_template("Tell me a curious fact about {historical_figure}.")

In [11]:
chain = prompt | chatModel.bind(stop=["Israel"]) | StrOutputParser()

In [12]:
result = chain.invoke({"historical_figure": "Albert Einstein"})
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 11 Jan 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Tell me a curious fact about Albert Einstein.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

One fascinating fact about Albert Einstein is that he was a passionate violinist and played the violin in his spare time. In fact, he was quite skilled and even performed with the Vienna Symphony Orchestra at the age of 17.


# Combine chain


In [13]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-3.5-turbo")

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a sentence about {politician}")

chain = prompt | chatModel | StrOutputParser()

In [15]:
chain.invoke("rahulgandhi")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me a sentence about rahulgandhi<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nRahul Gandhi, also known as Rahul Gandhi, is an Indian politician and the current President of the Indian National Congress party.'

In [16]:
historian_prompt = ChatPromptTemplate.from_template("Was {politician} positive for Humanity?")

In [17]:
composed_chain = {"politician": chain} | historian_prompt | model | StrOutputParser()

In [19]:
composed_chain.invoke({"politician": "rahulgandhi"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/

'Human: Was <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntell me a sentence about rahulgandhi<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nRahul Gandhi, also known as Rahul Gandhi, is an Indian politician and the current President of the Indian National Congress party. positive for Humanity?'

In [20]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI



prompt1 = ChatPromptTemplate.from_template("what is the country {politician} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what continent is the country {country} in? respond in {language}"
)


# model = ChatOpenAI()

chain1 = prompt1 | chatModel | StrOutputParser()

In [21]:
chain2 = (
    {"country": chain1, "language": itemgetter("language")}
    | prompt2
    | chatModel
    | StrOutputParser()
)

In [22]:
chain2.invoke({"politician": "trump", "language": "bhojpuri"})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwhat continent is the country <|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 11 Jan 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwhat is the country trump is from?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDonald Trump, the 45th President of the United States, was born in Queens, New York City, USA. in? respond in bhojpuri<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI can respond in Hindi'

# RAG Applicaiton


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [ ]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

/Users/myhome/Downloads/LANG/lang/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique where complex tasks are broken down into smaller and simpler steps to make them more manageable. It can be done using prompting techniques like Chain of Thought or Tree of Thoughts to guide the model in decomposing tasks effectively. Task decomposition can be facilitated by the use of LLM with simple prompts, task-specific instructions, or human inputs.'